### Task 목표 - Faster R-CNN backbone 선택 시 주의 사항 파악
#### 1) 관측 결과(사실)
- (현재 세팅 기준) VGG16 backbone + Faster R-CNN에서
- 1 epoch(750 iters) ≈ 12시간 소요 → 5 epoch ≈ 60시간(약 2일 12시간)
- 반면 EfficientNet-B0 backbone + Faster R-CNN 실험은 상대적으로 빠르게 학습이 진행되었고, AP50 성능도 0.75 이상 도달함(예: epoch 14에서 AP50≈0.762).
#### 2) 결론(판단)
- 동일 자원/환경에서 빠른 실험 반복이 필요한 상황이라면, VGG16을 Faster R-CNN backbone으로 쓰는 것은 비용 대비 비효율적일 가능성이 큼.
- EfficientNet 계열 backbone이 속도/성능 면에서 유리한 후보가 될 수 있음.
### EfficientNet이 유리했던 이유(가설/원인 후보 — 추가 검증 필요)
#### (가설 1) AMP 사용 여부 차이
##### EfficientNet 실험 코드에는 AMP가 포함됨:
- USE_AMP = (DEVICE.type == "cuda")
- with torch.amp.autocast("cuda", enabled=USE_AMP):
- scaler = torch.amp.GradScaler(...)
##### VGG 실험 코드에는 AMP가 없었음 → 혼합정밀 적용 유무가 iteration time에 큰 차이를 만들었을 가능성.
#### (가설 2) 데이터 로딩/디코딩 방식 차이
##### EfficientNet 코드: Dataset 로딩 시 이미지를 미리 열어(PIL) 메모리에 보관(preload)
- image = Image.open(img_path).convert("RGB")
- data.append((image, target))
- VGG 코드: __getitem__에서 매 iteration마다 이미지 open/디코딩(lazy loading)
##### → (특히 num_workers=0 환경에서) CPU 디코딩/IO 병목으로 GPU 대기 시간이 증가했을 가능성.
#### (가설 3) backbone 아키텍처 효율 차이
##### EfficientNet-B0는 연산 효율이 높은 구조(모바일/경량 지향)인 반면
##### VGG16은 구형 구조로 연산 효율이 낮은 편
##### → 동일 detector에서 step당 연산량 차이가 누적되었을 가능성.

In [1]:
# ============================================================
# PyTorch 2.6.0+cu126 대응 버전
# Faster R-CNN (GPU) + EfficientNet-B0 backbone
# 목표: COCOeval AP@0.50(AP50) 기준
#  - AP50 >= 0.75 : pass 모델 저장(계속 학습)
#  - AP50 >= 0.80 : 모델 저장 + 즉시 종료
#
# DataLoader: num_workers=0 (Windows/Jupyter 안정)
# ============================================================

import os
import numpy as np
import torch
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models, ops
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import rpn

In [3]:
# ---------------------------
# 0) 기본 설정
# ---------------------------
ROOT = "../datasets/coco"     # <- 데이터 루트
BATCH_SIZE = 4
NUM_WORKERS = 0              # <- 요청하신 대로 0
EPOCHS = 120
EVAL_EVERY = 2
TARGET_AP50 = 0.75
STOP_AP50 = 0.80
PATIENCE = 12

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = (DEVICE.type == "cuda")

print("Torch:", torch.__version__)
print("DEVICE:", DEVICE)

Torch: 2.6.0+cu126
DEVICE: cuda


In [5]:
# ---------------------------
# 1) Dataset (COCO -> contiguous label 매핑 포함)
# ---------------------------
class COCODataset(Dataset):
    def __init__(self, root, train: bool, transform=None):
        super().__init__()
        split = "train" if train else "val"
        ann_path = os.path.join(root, "annotations", f"{split}_annotations.json")
        self.coco = COCO(ann_path)
        self.image_dir = os.path.join(root, split)
        self.transform = transform

        # category_id(원본) -> contiguous(1..K), 0은 background
        cat_ids = sorted(list(self.coco.cats.keys()))
        self.cat_id_to_contiguous = {cat_id: i + 1 for i, cat_id in enumerate(cat_ids)}
        self.contiguous_to_cat_id = {v: k for k, v in self.cat_id_to_contiguous.items()}

        # 표시용: contiguous -> name
        self.categories = {0: "background"}
        for cat_id in cat_ids:
            self.categories[self.cat_id_to_contiguous[cat_id]] = self.coco.cats[cat_id]["name"]

        self.data = self._load_data()

    def _load_data(self):
        data = []
        for img_id in self.coco.imgs:
            img_info = self.coco.loadImgs(img_id)[0]
            file_name = img_info["file_name"]
            img_path = os.path.join(self.image_dir, file_name)
            image = Image.open(img_path).convert("RGB")

            anns = self.coco.loadAnns(self.coco.getAnnIds(img_id))
            boxes, labels = [], []
            for ann in anns:
                if ann.get("iscrowd", 0) == 1:
                    continue
                x, y, w, h = ann["bbox"]  # COCO: [x,y,w,h]
                if w <= 0 or h <= 0:
                    continue
                boxes.append([x, y, x + w, y + h])  # -> xyxy
                labels.append(self.cat_id_to_contiguous[int(ann["category_id"])])

            target = {
                "image_id": torch.tensor([img_id], dtype=torch.int64),
                "boxes": torch.tensor(boxes, dtype=torch.float32),
                "labels": torch.tensor(labels, dtype=torch.int64),
            }
            data.append((image, target))
        return data

    def __getitem__(self, idx):
        image, target = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, target

    def __len__(self):
        return len(self.data)


def collator(batch):
    return tuple(zip(*batch))

In [7]:
# ---------------------------
# 2) Transform / DataLoader (num_workers=0)
# ---------------------------
transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(dtype=torch.float),
])

train_ds = COCODataset(ROOT, train=True, transform=transform)
val_ds   = COCODataset(ROOT, train=False, transform=transform)

PIN_MEMORY = torch.cuda.is_available()

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=collator,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)
val_loader = DataLoader(
    val_ds,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    collate_fn=collator,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [9]:
# ---------------------------
# 3) Model: Faster R-CNN + EfficientNet-B0 backbone
# ---------------------------
# EfficientNet-B0 features out_channels = 1280
backbone = models.efficientnet_b0(
    weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1
).features
backbone.out_channels = 1280

# 작은 객체에 유리하도록 anchor에 작은 size 포함
anchor_generator = rpn.AnchorGenerator(
    sizes=((16, 32, 64, 128, 256),),
    aspect_ratios=((0.5, 1.0, 2.0),),
)

roi_pooler = ops.MultiScaleRoIAlign(
    featmap_names=["0"],     # backbone이 단일 feature map을 반환한다고 가정
    output_size=(7, 7),
    sampling_ratio=2,
)

num_classes = 1 + len(train_ds.cat_id_to_contiguous)  # background + K
model = FasterRCNN(
    backbone=backbone,
    num_classes=num_classes,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler,
).to(DEVICE)

In [11]:
# ---------------------------
# 4) Optimizer / Scheduler / AMP
# ---------------------------
from torch import optim

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.002, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

# PyTorch 2.6 권장 AMP API
scaler = torch.amp.GradScaler("cuda", enabled=USE_AMP)

In [18]:
# ---------------------------
# 5) COCOeval AP50 평가 함수 (학습 중엔 summarize 출력 X)
# ---------------------------
import io
import contextlib
import numpy as np
from pycocotools.cocoeval import COCOeval

def evaluate_ap50(
    model,
    dataloader,
    dataset,
    verbose: bool = False,
    topk: int = 300,           # ✅ 이미지당 상위 N개만 평가(속도 핵심)
    score_thresh: float = 0.0  # ✅ 필요시 0.05~0.2 등으로 올리면 더 빨라짐
) -> float:
    """
    Faster R-CNN 출력 -> COCOeval AP50 계산
    - 속도 최적화:
      1) score_thresh로 1차 필터
      2) topk로 이미지당 예측 개수 제한
    - summarize는 stats 채우기 위해 항상 호출하되, verbose=False면 출력만 숨김
    """
    model.eval()
    coco_dets = []

    with torch.no_grad():
        for images, targets in dataloader:
            images = [img.to(DEVICE, non_blocking=True) for img in images]
            outputs = model(images)

            for i in range(len(targets)):
                image_id = int(targets[i]["image_id"].cpu().numpy()[0])

                boxes = outputs[i]["boxes"].detach().cpu().numpy()    # xyxy
                scores = outputs[i]["scores"].detach().cpu().numpy()
                labels = outputs[i]["labels"].detach().cpu().numpy()  # contiguous

                if boxes.shape[0] == 0:
                    continue

                # 1) score threshold 필터
                if score_thresh > 0.0:
                    keep = scores >= score_thresh
                    boxes = boxes[keep]
                    scores = scores[keep]
                    labels = labels[keep]
                    if boxes.shape[0] == 0:
                        continue

                # 2) topk 제한 (점수 내림차순)
                if topk is not None and boxes.shape[0] > topk:
                    order = np.argsort(-scores)[:topk]
                    boxes = boxes[order]
                    scores = scores[order]
                    labels = labels[order]

                # xyxy -> xywh
                boxes_xywh = boxes.copy()
                boxes_xywh[:, 2] = boxes_xywh[:, 2] - boxes_xywh[:, 0]
                boxes_xywh[:, 3] = boxes_xywh[:, 3] - boxes_xywh[:, 1]

                # detections append
                for j in range(len(boxes_xywh)):
                    cont = int(labels[j])
                    coco_cat_id = int(dataset.contiguous_to_cat_id[cont])
                    coco_dets.append([
                        image_id,
                        float(boxes_xywh[j, 0]), float(boxes_xywh[j, 1]),
                        float(boxes_xywh[j, 2]), float(boxes_xywh[j, 3]),
                        float(scores[j]),
                        coco_cat_id
                    ])

    # 탐지 결과가 아예 없으면 AP50=0
    if len(coco_dets) == 0:
        return 0.0

    coco_dets = np.asarray(coco_dets)
    coco_gt = dataset.coco
    coco_dt = coco_gt.loadRes(coco_dets)

    coco_eval = COCOeval(coco_gt, coco_dt, iouType="bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()

    # stats 채우기 위해 summarize는 항상 호출 (출력만 제어)
    if verbose:
        coco_eval.summarize()
    else:
        with contextlib.redirect_stdout(io.StringIO()):
            coco_eval.summarize()

    # AP@[IoU=0.50] = stats[1]
    return float(coco_eval.stats[1])

In [20]:
# ---------------------------
# 6) Train: AP50만 출력, 0.75 저장, 0.80 종료, patience 종료
# ---------------------------
best_ap50 = -1.0
bad_count = 0
pass_saved = False

for epoch in range(1, EPOCHS + 1):
    model.train()
    running = 0.0

    for images, targets in train_loader:
        images = [img.to(DEVICE, non_blocking=True) for img in images]
        targets = [{k: v.to(DEVICE, non_blocking=True) for k, v in t.items()} for t in targets]

        optimizer.zero_grad(set_to_none=True)

        # PyTorch 2.6 권장 autocast API
        with torch.amp.autocast("cuda", enabled=USE_AMP):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()

        running += float(losses.detach().item())

    lr_scheduler.step()

    # ----- evaluation -----
    if epoch % EVAL_EVERY != 0:
        continue

    ap50 = evaluate_ap50(model, val_loader, val_ds, verbose=False)
    print(f"epoch={epoch:03d}  AP50={ap50:.4f}")

    # 0.75: 통과 저장만 (멈추지 않음)
    if (ap50 >= TARGET_AP50) and (not pass_saved):
        torch.save(model.state_dict(), "pass_ap50_0.75.pt")
        pass_saved = True

    # 0.80: 저장 + 즉시 종료
    if ap50 >= STOP_AP50:
        torch.save(model.state_dict(), "pass_ap50_0.80.pt")
        print("🎯 STOP(AP50>=0.80) reached -> saved & stop")
        break

    # best 갱신 + patience (0.80 못 가면 여기서 종료될 수 있음)
    if ap50 > best_ap50:
        best_ap50 = ap50
        bad_count = 0
        torch.save(model.state_dict(), "best_ap50.pt")
    else:
        bad_count += 1
        if bad_count >= PATIENCE:
            print("🛑 Early stopping (no improvement)")
            break

print("Done.")

Loading and preparing results...
Converting ndarray to lists...
(827, 7)
0/827
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.03s).
epoch=002  AP50=0.5934
Loading and preparing results...
Converting ndarray to lists...
(658, 7)
0/658
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.02s).
epoch=004  AP50=0.6673
Loading and preparing results...
Converting ndarray to lists...
(794, 7)
0/794
DONE (t=0.10s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.02s).
epoch=006  AP50=0.6896
Loading and preparing results...
Converting ndarray to lists...
(656, 7)
0/656
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation..

In [28]:
def pretty_print_result(coco_eval, target_ap50=0.75):
    ap, ap50, ap75, aps, apm, apl = coco_eval.stats[:6]

    status = "PASS" if ap50 >= target_ap50 else "FAIL"

    print("\n" + "="*40)
    print("COCO Evaluation Summary")
    print("="*40)

    print(f"Status        : {status}")
    print(f"Target AP50   : {target_ap50:.2f}")
    print("-"*40)

    print(f"AP50          : {ap50:.3f} ({ap50*100:.1f}%)")
    print(f"AP75          : {ap75:.3f} ({ap75*100:.1f}%)")
    print(f"AP@[0.50:0.95]: {ap:.3f} ({ap*100:.1f}%)")

    print("-"*40)
    print(f"Small Object  : {aps:.3f}")
    print(f"Medium Object : {apm:.3f}")
    print(f"Large Object  : {apl:.3f}")
    print("="*40 + "\n")

In [22]:
# ---------------------------
# 7) (선택) 학습 후 공식 표(12줄) 출력은 여기서만
#    - 필요할 때만 아래 두 줄을 실행하세요.
# ---------------------------
model.load_state_dict(torch.load("best_ap50.pt", map_location=DEVICE))
_ = evaluate_ap50(model, val_loader, val_ds, verbose=True)

Loading and preparing results...
Converting ndarray to lists...
(663, 7)
0/663
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.374
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.771
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.294
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.425
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.444
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.510
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.510
 Average Recall     (A

In [36]:
import io, contextlib
import numpy as np
from pycocotools.cocoeval import COCOeval

# 1) 예쁘게 출력 함수 (결과만)
def pretty_print_result(coco_eval, target_ap50=0.75):
    ap, ap50, ap75, aps, apm, apl = coco_eval.stats[:6]
    status = "PASS" if ap50 >= target_ap50 else "❌ FAIL"

    print("\n" + "="*44)
    print(f"{status}  (Target AP50 ≥ {target_ap50:.2f})")
    print("-"*44)
    print(f"AP50          : {ap50:.3f} ({ap50*100:.1f}%)")
#    print(f"AP75          : {ap75:.3f} ({ap75*100:.1f}%)")
#    print(f"mAP@[.50:.95] : {ap:.3f} ({ap*100:.1f}%)")
    print("-"*44)
    print(f"AP small/med/large : {aps:.3f} / {apm:.3f} / {apl:.3f}")
    print("="*44 + "\n")

# 2) COCOeval "객체"를 만들어주는 평가 함수 (훈련X)
def build_coco_eval(
    model, dataloader, dataset,
    topk=300, score_thresh=0.0, silent=True
):
    model.eval()
    coco_dets = []

    with torch.no_grad():
        for images, targets in dataloader:
            images = [img.to(DEVICE, non_blocking=True) for img in images]
            outputs = model(images)

            for i in range(len(targets)):
                image_id = int(targets[i]["image_id"].cpu().numpy()[0])

                boxes = outputs[i]["boxes"].detach().cpu().numpy()
                scores = outputs[i]["scores"].detach().cpu().numpy()
                labels = outputs[i]["labels"].detach().cpu().numpy()

                if boxes.shape[0] == 0:
                    continue

                # threshold
                if score_thresh > 0.0:
                    keep = scores >= score_thresh
                    boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
                    if boxes.shape[0] == 0:
                        continue

                # topk
                if topk is not None and boxes.shape[0] > topk:
                    order = np.argsort(-scores)[:topk]
                    boxes, scores, labels = boxes[order], scores[order], labels[order]

                # xyxy -> xywh
                boxes_xywh = boxes.copy()
                boxes_xywh[:, 2] -= boxes_xywh[:, 0]
                boxes_xywh[:, 3] -= boxes_xywh[:, 1]

                for j in range(len(boxes_xywh)):
                    coco_cat_id = int(dataset.contiguous_to_cat_id[int(labels[j])])
                    coco_dets.append([
                        image_id,
                        float(boxes_xywh[j, 0]), float(boxes_xywh[j, 1]),
                        float(boxes_xywh[j, 2]), float(boxes_xywh[j, 3]),
                        float(scores[j]),
                        coco_cat_id
                    ])

    coco_dets = np.asarray(coco_dets)
    coco_gt = dataset.coco
    coco_dt = coco_gt.loadRes(coco_dets)

    coco_eval = COCOeval(coco_gt, coco_dt, iouType="bbox")

    if silent:
        with contextlib.redirect_stdout(io.StringIO()):
            coco_eval.evaluate()
            coco_eval.accumulate()
            coco_eval.summarize()   # stats 채우기용(출력은 숨김)
    else:
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

    return coco_eval


# --------------------------
# 실행 (훈련 아님)
# --------------------------
# best 모델 불러오기
model.load_state_dict(torch.load("best_ap50.pt", map_location=DEVICE))

# 평가 1회로 coco_eval 만들기
coco_eval = build_coco_eval(
    model, val_loader, val_ds,
    topk=300, score_thresh=0.0, silent=True
)

# 결과출력
pretty_print_result(coco_eval, target_ap50=0.75)

Loading and preparing results...
Converting ndarray to lists...
(663, 7)
0/663
DONE (t=0.00s)
creating index...
index created!

PASS  (Target AP50 ≥ 0.75)
--------------------------------------------
AP50          : 0.771 (77.1%)
--------------------------------------------
AP small/med/large : 0.043 / 0.330 / 0.425

